In [64]:
import cv2
import torch
import torchvision.models as models
from torchvision.transforms import v2
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams
from qdrant_client.models import PointStruct
import hashlib
import time
import torch.nn as nn
import torch.nn.functional as F

## Qdrant init. Run the qdrant client first

docker pull qdrant/qdrant


docker run -p 6333:6333 -p 6334:6334 \
    -v $(pwd)/qdrant_storage:/qdrant/storage:z \
    qdrant/qdrant

In [86]:
client = QdrantClient(url="http://localhost:6333")

video_emb_dim = 500
distance = Distance.EUCLID

client.delete_collection('video')
client.create_collection(
    collection_name="video",
    vectors_config=VectorParams(size=video_emb_dim, distance=distance)
)

THRESHOLD = 0.677

## Code for obtaining video vector representation from alexnet

In [87]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Load AlexNet model and extract convolutional layers
alexnet = models.alexnet(weights="AlexNet_Weights.DEFAULT").to(device)
conv_layers = nn.Sequential(*list(alexnet.features.children()))
alexnet.eval()

# Define transformations
transforms = v2.Compose([
    v2.Resize(size=(224, 224), antialias=True),
    v2.ToTensor(),
    v2.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])  

In [88]:
def retrieve_frames(video_path, start_time, fps=1):
    """
    Retrieves frames from a video at a specified frame rate between a start and end time
    and returns them as a list of torch tensors.

    :param video_path: Path to the video file
    :param start_time: Start time in seconds
    :param end_time: End time in seconds
    :param fps: Frame rate at which to extract frames (default is 2 fps)
    :return: List of frames as NumPy ndarrays
    """
    # Open the video file
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Error: Could not open video file {video_path}")
        return torch.empty()

    # Get video properties
    video_fps = cap.get(cv2.CAP_PROP_FPS)
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    total_duration = frame_count / video_fps

    # Convert start and end times to frame numbers
    start_frame = int(start_time * video_fps)
    end_frame = total_duration

    # Set the video capture to the start frame
    cap.set(cv2.CAP_PROP_POS_FRAMES, start_frame)

    # Calculate the interval between frames to extract
    interval = int(video_fps / fps)

    # Extract frames
    frames = []
    frame_number = start_frame
    while cap.isOpened() and frame_number <= end_frame:
        ret, frame = cap.read()
        if not ret:
            break

        # Append the frame to the list
        frame = torch.from_numpy(frame).permute(2, 0, 1) / 255
        frames.append(transforms(frame))

        # Move to the next frame to extract
        frame_number += interval
        cap.set(cv2.CAP_PROP_POS_FRAMES, frame_number)

    # Release the video capture object
    cap.release()
    frames = torch.stack(frames)
    return frames.detach().cpu()

class MaxPoolConvOutputs(nn.Module):
    def __init__(self, conv_layers):
        super(MaxPoolConvOutputs, self).__init__()
        self.conv_layers = conv_layers
        self.pool = nn.AdaptiveMaxPool2d((1, 1))  # Pool to a single value per channel

    def forward(self, x):
        outputs = []
        for layer in self.conv_layers:
            x = layer(x)
            if isinstance(layer, nn.Conv2d):  # Apply max pooling only on Conv2d outputs
                pooled_output = self.pool(x).squeeze(-1).squeeze(-1)  # Remove spatial dims
                outputs.append(pooled_output)
        return outputs

def get_frames_emb(video):
    max_pool_extractor = MaxPoolConvOutputs(conv_layers)

    with torch.no_grad():
        max_pooled_outputs = max_pool_extractor(video.to(device))

    result = torch.cat(max_pooled_outputs, dim=1)

    return result

def normalize_frames(video):
    # Step 1: Average across frames (mean along the first dimension)
    avg_emb = torch.mean(video, dim=0)

    # Step 2: Zero-mean normalization (subtract the mean of the vector)
    mean_value = torch.mean(avg_emb)
    zero_mean_emb = avg_emb - mean_value

    # Step 3: ℓ2-normalization (normalize by the L2 norm)
    l2_norm = torch.norm(zero_mean_emb, p=2)
    l2_normalized_emb = zero_mean_emb / l2_norm

    return l2_normalized_emb


class EmbeddingNet(nn.Module):
    def __init__(self):
        super(EmbeddingNet, self).__init__()
        self.fc1 = nn.Linear(1152, 1500)  # Assuming input images are 28x28
        self.fc2 = nn.Linear(1500, 1000)
        self.fc3 = nn.Linear(1000, 500)  # Output embedding of size 500

    def forward(self, x):
        x = x.view(x.size(0), -1)  # Flatten the input
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)  # Output embedding
        x = F.normalize(x, p=2, dim=1)  # Normalize embeddings to have unit norm
        return x

base_model = EmbeddingNet().to(device)
base_model.load_state_dict(torch.load('api/model_77.pt', map_location=device))
base_model.eval()

/tmp/ipykernel_43142/2542842698.py:110: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  base_model.load_state_dict(torch.load('api/model_77.pt', map_location=device))


EmbeddingNet(
  (fc1): Linear(in_features=1152, out_features=1500, bias=True)
  (fc2): Linear(in_features=1500, out_features=1000, bias=True)
  (fc3): Linear(in_features=1000, out_features=500, bias=True)
)

## The main pipeline

In [89]:
def main(video_file_path):
    # Get video descriptor
    frames = retrieve_frames(video_file_path, start_time=0, fps=2)
    emb = get_frames_emb(frames)
    embedding_tensor = normalize_frames(emb)
    
    # Project the video descriptor into the embedding space
    with torch.no_grad():
        vector = base_model(embedding_tensor.unsqueeze(0).to(device)).detach().cpu().numpy()[0]
        
    # Search for the closest video in a collection
    search_result = client.query_points(
        collection_name="video",
        query=vector,
        with_payload=True,
        limit=1
    ).points
    
    # If the result is not empty, check for the threshold
    if len(search_result) > 0:
        name2 = search_result[0].payload['name']
        if search_result[0].score < THRESHOLD:
            print(f"Duplicate found: {name2}, score: {search_result[0].score}")
            return True
        else:
            print(f"Duplicate not found, closest score with {name2}: {search_result[0].score}")

    # Insert a video if it is not a duplicate
    current_timestamp = str(time.time())
    hashed_hex = hashlib.sha256(current_timestamp.encode()).hexdigest()
    hashed_id = int(hashed_hex, 16) % (2**64)
    client.upsert(
        collection_name="video",
        points=[PointStruct(id=hashed_id, vector=vector, payload={'name': f'video_{hashed_id}'})]
    )
    print(f"New video uploaded: video_{hashed_id}")
    return False 
    

### Insert a new video

In [90]:
video_path = "test_data/orig.mp4"
main(video_path)

New video uploaded: video_15791415937813033677


False

### Insert a duplicate. This video is resized and in a lower resolution

In [91]:
video_path = "test_data/dup.mp4"
main(video_path)

Duplicate found: video_15791415937813033677, score: 0.48300344


True

### Insert a negative video

In [92]:
video_path = "test_data/negative.mp4"
main(video_path)

Duplicate not found, closest score with video_15791415937813033677: 1.5475478
New video uploaded: video_7238750216663548295


False

Note that videos embeddings are stored in a vector database. You may want to reset it with `client.delete_collection('video')`